# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [31]:
# Write your code below.
import pandas as pd
import os
import sys
from glob import glob
import dask.dataframe as dd

from dotenv import load_dotenv



In [32]:
load_dotenv(override=True) 

True

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [33]:
# CLean up before start
PRICE_DATA = os.getenv("PRICE_DATA")
print(PRICE_DATA)

../../05_src/data/prices/


In [34]:
full_path = os.path.join(PRICE_DATA, "**/*.parquet")
#full_path = "../../05_src/data/prices/**/*.parquet"
parquet_files = glob(full_path, recursive = True)
print(full_path)
print(parquet_files)

../../05_src/data/prices/**/*.parquet
['../../05_src/data/prices/DENN/DENN_1998/part.1.parquet', '../../05_src/data/prices/DENN/DENN_1998/part.0.parquet', '../../05_src/data/prices/DENN/DENN_2010/part.1.parquet', '../../05_src/data/prices/DENN/DENN_2010/part.0.parquet', '../../05_src/data/prices/DENN/DENN_1999/part.1.parquet', '../../05_src/data/prices/DENN/DENN_1999/part.0.parquet', '../../05_src/data/prices/DENN/DENN_2000/part.1.parquet', '../../05_src/data/prices/DENN/DENN_2000/part.0.parquet', '../../05_src/data/prices/DENN/DENN_2020/part.1.parquet', '../../05_src/data/prices/DENN/DENN_2020/part.0.parquet', '../../05_src/data/prices/DENN/DENN_2005/part.1.parquet', '../../05_src/data/prices/DENN/DENN_2005/part.0.parquet', '../../05_src/data/prices/DENN/DENN_2007/part.1.parquet', '../../05_src/data/prices/DENN/DENN_2007/part.0.parquet', '../../05_src/data/prices/DENN/DENN_2006/part.1.parquet', '../../05_src/data/prices/DENN/DENN_2006/part.0.parquet', '../../05_src/data/prices/DENN/DE

In [35]:
dd_px = dd.read_parquet(parquet_files).set_index('ticker')

In [36]:
print(dd_px)

Dask DataFrame Structure:
                          Date     Open     High      Low    Close Adj Close Volume  source   Year
npartitions=90                                                                                    
ACU             datetime64[ns]  float64  float64  float64  float64   float64  int64  string  int32
ANF                        ...      ...      ...      ...      ...       ...    ...     ...    ...
...                        ...      ...      ...      ...      ...       ...    ...     ...    ...
YCBD                       ...      ...      ...      ...      ...       ...    ...     ...    ...
YCBD                       ...      ...      ...      ...      ...       ...    ...     ...    ...
Dask Name: setindex, 2 expressions
Expr=SetIndex(frame=ReadParquetFSSpec(e713907), _other='ticker', options={})


In [37]:
print(dd_px.columns)

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'source',
       'Year'],
      dtype='object')


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [38]:
# Ensure data is sorted by Ticker and Date
df = dd_px.sort_values(['ticker'])

In [39]:

# Define function to apply per Ticker group
def compute_features(group):
    group = group.set_index('Date', sorted=False)  # Ensure proper indexing for lags
    group['Close_lag_1'] = group['Close'].shift(1)
    group['Adj_Close_lag_1'] = group['Adj_Close'].shift(1)
    group['returns'] = (group['Close'] / group['Close_lag_1']) - 1
    group['hi_lo_range'] = group['High'] - group['Low']
    return group.reset_index()

In [40]:
# Apply feature computation per Ticker group
dd_feat = df.groupby('ticker').apply(
    compute_features,
    meta={
        'Date': 'datetime64[ns]',
        'ticker': 'object',
        'Close': 'float64',
        'Adj_Close': 'float64',
        'High': 'float64',
        'Low': 'float64',
        'Close_lag_1': 'float64',
        'Adj_Close_lag_1': 'float64',
        'returns': 'float64',
        'hi_lo_range': 'float64',
    }
)

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [42]:
# Write your code below.
pdf = dd_feat.compute()

# Add a new column with the 10-day moving average of 'returns'
pdf['returns_ma_10'] = pdf['returns'].rolling(window=10).mean()

KeyError: 'ticker'

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

Yes, it was necessary because the .rolling() method is only available on pandas DataFrames and Series, not directly on Dask DataFrames.
+ Would it have been better to do it in Dask? Why?

if the data is large - Dask would be better because it can split data in chunks and perform calculations in parallel
 
(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.